In [ ]:
%tensorflow_version 1.13.2

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.13.2`. This will be interpreted as: `1.x`.


TensorFlow is already loaded. Please restart the runtime to change versions.


In [ ]:
!nvidia-smi

Thu Mar 11 11:48:14 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    29W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#colab連接到GOOGLE Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# create target directory
! rm -rf /content/labels
! rm -rf /content/images
! mkdir /content/labels
! mkdir /content/images

In [ ]:
!tar -xf /content/drive/Shareddrives/AI/train/JEPGImages/part1.tar -C /content/images

In [ ]:
!tar -xf /content/drive/Shareddrives/AI/train/JEPGImages/part2.tar -C /content/images

In [ ]:
!tar -xf /content/drive/Shareddrives/AI/train/JEPGImages/part3.tar -C /content/images

In [ ]:
!tar -xf /content/drive/Shareddrives/AI/train/Annotations/Annotations.tar -C /content/labels/

In [ ]:
# Create a xml to yolo format converter
# additionally, include remove functions to save disk space
from bs4 import BeautifulSoup
from pathlib import Path
import os
import shutil

#建置一個status_dic
status_dic = {'bicycle':3, 'scooter':2, 'pedestrian':1, 'vehicle':0}

def getYoloFormat(filename,label_path, img_path):
    with open(label_path + filename, 'r') as f:
        soup = BeautifulSoup(f.read(), 'xml')
        imgname = Path(soup.select_one('filename').text).stem + '.jpg'
        # image_w = int(soup.select_one('width').text)
        image_w = 1920
        # image_h = int(soup.select_one('height').text)
        image_h = 1080
        ary = []
        for obj in soup.select('object'):
            #取出xmin, xmax, ymin, ymax，並用status_dic轉換
            xmin = int(obj.select_one('xmin').text)
            xmax = int(obj.select_one('xmax').text)
            ymin = int(obj.select_one('ymin').text)
            ymax = int(obj.select_one('ymax').text)
            objclass = status_dic.get(obj.select_one('name').text)

            #轉成YOLO格式
            x = (xmin + (xmax-xmin)/2)*1.0/image_w
            y = (ymin + (ymax-ymin)/2)*1.0/image_h
            w = (xmax-xmin)*1.0/image_w
            h = (ymax-ymin)*1.0/image_h
            ary.append(' '.join([str(objclass), str(x), str(y), str(w), str(h)]))

        #remove original xml
        os.remove(label_path + filename)


        if os.path.exists(img_path + imgname):
            #把圖片移到yolo之下，同時把參數寫到yolo之下
            # shutil.copyfile(img_path + imgname, yolo_path)
            with open(label_path + Path(imgname).stem + '.txt', 'w') as f:
              f.write('\n'.join(ary))
        
        #remove original image
        # os.remove(img_path + imgname)

In [ ]:
import os, shutil, sys 

def move_to_root_folder(root_path, cur_path):
    for filename in os.listdir(cur_path):
        if os.path.isfile(os.path.join(cur_path, filename)):
            shutil.move(os.path.join(cur_path, filename), os.path.join(root_path, filename))
        elif os.path.isdir(os.path.join(cur_path, filename)):
            move_to_root_folder(root_path, os.path.join(cur_path, filename))
        else:
            sys.exit("Should never reach here.")
    # remove empty folders
    if cur_path != root_path:
        os.rmdir(cur_path)

In [ ]:
import os
#設定路徑
labelpath = '/content/labels/'
imgpath = '/content/images/'

move_to_root_folder(labelpath,labelpath)
move_to_root_folder(imgpath,imgpath)

NameError: ignored

In [ ]:
import os
#設定路徑
labelpath = '/content/labels/'
imgpath = '/content/images/'
# yolopath = '/content/yolo/'
# ary = []
for idx, f in enumerate(os.listdir(labelpath)):
    try:
        #透過getYoloFromat將圖像和參數檔寫到YOLO下
        getYoloFormat(f, labelpath, imgpath)
    except Exception as e:
        print(e)

'utf-8' codec can't decode byte 0xaf in position 16: invalid start byte
'utf-8' codec can't decode byte 0x99 in position 16: invalid start byte
'utf-8' codec can't decode byte 0xca in position 18: invalid continuation byte


In [ ]:
import os
datasets = ['/content/images/' + f for f in os.listdir('/content/images/') if f.endswith('.jpg')]
round(len(datasets))

89002

In [ ]:
import os
datasets = ['/content/labels/' + f for f in os.listdir('/content/labels/') if f.endswith('.txt')]
round(len(datasets))

89002

In [ ]:
! rm -rf /content/labels/train
! rm -rf /content/images/train
! mkdir /content/labels/train
! mkdir /content/images/train
! rm -rf /content/labels/validation
! rm -rf /content/images/validation
! mkdir /content/labels/validation
! mkdir /content/images/validation

In [ ]:
import pandas as pd
from sklearn import model_selection
df = datasets
df_train,df_valid=model_selection.train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    shuffle=True
)
print(df_train[0])
# type(df_train)
print(len(df_train))
print(len(df_valid))

/content/labels/2_78_00.mp4_01488.txt
71201
17801


In [ ]:
import shutil
from pathlib import Path

for i in df_train:
  shutil.move(i,'/content/labels/train')
  shutil.move('/content/images/'+Path(i).stem+'.jpg','/content/images/train')

for i in df_valid:
  shutil.move(i,'/content/labels/validation')
  shutil.move('/content/images/'+Path(i).stem+'.jpg','/content/images/validation')

In [ ]:
#建立YOLO資料夾
! rm -rf /content/yolo
! mkdir /content/yolo

In [ ]:
import shutil
shutil.move('/content/images','/content/yolo')
shutil.move('/content/labels','/content/yolo')

'/content/yolo/labels'

In [ ]:
import os
count=[f for f in os.listdir('/content/yolo/labels/train') if f.endswith('.txt')]
len(count)

71201

In [ ]:
import os
count=[f for f in os.listdir('/content/yolo/labels/validation') if f.endswith('.txt')]
len(count)

17801

In [ ]:
import os
count=[f for f in os.listdir('/content/yolo/images/train') if f.endswith('.jpg')]
len(count)

71201

In [ ]:
import os
count=[f for f in os.listdir('/content/yolo/images/validation') if f.endswith('.jpg')]
len(count)

17801

In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
%cd /content/drive/MyDrive/yolov5/

/content/drive/MyDrive/yolov5


In [ ]:
if not os.path.exists('yolov5/'):
  !git clone https://github.com/avBuffer/Yolov5_tf.git

Cloning into 'Yolov5_tf'...
remote: Enumerating objects: 291, done.
remote: Counting objects: 100% (291/291), done.
remote: Compressing objects: 100% (214/214), done.
remote: Total 291 (delta 99), reused 242 (delta 56), pack-reused 0
Receiving objects: 100% (291/291), 23.74 MiB | 21.92 MiB/s, done.
Resolving deltas: 100% (99/99), done.
Checking out files: 100% (129/129), done.


In [ ]:
# This stops 'Run all' at this cell by causing an error
assert False
# remember to adjust the number of classes (nc) in the yolov5 yaml file
# for example: yolov5/models/yolov5s.yaml

AssertionError: ignored

In [ ]:
%pwd

'/content/drive/MyDrive/yolov5'

In [ ]:
!cp /content/drive/Shareddrives/AI/yolov5l.pt /content

In [ ]:
# train from pre-trained 
!python train.py --data /content/drive/Shareddrives/AI/data.yaml --cfg /content/drive/MyDrive/yolov5/models/yolov5l.yaml --weights /content/drive/Shareddrives/AI/weights/yolov4-tiny_Ting-Yi/archive/last.pt --batch-size 24 --img 640 --epoch 3

remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Total 21 (delta 12), reused 12 (delta 12), pack-reused 9
Unpacking objects: 100% (21/21), done.
From https://github.com/ultralytics/yolov5
   ba18528..e8a2b83  master     -> origin/master
   5d90d0b..b140a2c  C6         -> origin/C6
github: ⚠️ WARNING: code is out of date by 8 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-115-ge931b9d torch 1.7.1+cu101 CUDA:0 (Tesla V100-SXM2-16GB, 16160.5MB)

Namespace(adam=False, batch_size=24, bucket='', cache_images=False, cfg='/content/drive/MyDrive/yolov5/models/yolov5l.yaml', data='/content/drive/Shareddrives/AI/data.yaml', device='', entity=None, epochs=3, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=Fa

In [ ]:
# train from last save
!python train.py --data /content/drive/Shareddrives/AI/data.yaml --cfg /content/drive/MyDrive/yolov5/models/yolov5m.yaml --weights /content/drive/MyDrive/yolov5/runs/train/exp2/weights/best.pt --batch-size 40 --img 640

github: ⚠️ WARNING: code is out of date by 4 commits. Use 'git pull' to update or 'git clone https://github.com/ultralytics/yolov5' to download latest.
YOLOv5 v4.0-115-ge931b9d torch 1.7.1+cu101 CUDA:0 (Tesla P100-PCIE-16GB, 16280.875MB)

Namespace(adam=False, batch_size=40, bucket='', cache_images=False, cfg='/content/drive/MyDrive/yolov5/models/yolov5m.yaml', data='/content/drive/Shareddrives/AI/data.yaml', device='', entity=None, epochs=300, evolve=False, exist_ok=False, global_rank=-1, hyp='data/hyp.scratch.yaml', image_weights=False, img_size=[640, 640], linear_lr=False, local_rank=-1, log_artifacts=False, log_imgs=16, multi_scale=False, name='exp', noautoanchor=False, nosave=False, notest=False, project='runs/train', quad=False, rect=False, resume=False, save_dir='runs/train/exp4', single_cls=False, sync_bn=False, total_batch_size=40, weights='/content/drive/MyDrive/yolov5/runs/train/exp2/weights/best.pt', workers=8, world_size=1)
wandb: Install Weights & Biases for YOLOv5 loggin

In [ ]:
!cp /content/drive/Shareddrives/AI/yolov5l.pt /content/drive/MyDrive/yolov5TF/

In [ ]:
! python /content/drive/MyDrive/yolov5TF/freeze_ckpt_to_pb.py

freeze_ckpt_to_pb ckpt_file= ckpts/social_yolov3_test-loss=3.2020.ckpt-198  not exist


In [ ]:
!git clone https://github.com/ultralytics/yolov5.git  # clone repo

Cloning into 'yolov5'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 5286 (delta 36), reused 44 (delta 22), pack-reused 5226
Receiving objects: 100% (5286/5286), 8.01 MiB | 13.91 MiB/s, done.
Resolving deltas: 100% (3607/3607), done.


In [ ]:
!pip install onnx>=1.8.0  # for ONNX export
!pip install coremltools==4.0  # for CoreML export
!pip install -r /content/yolov5/requirements.txt  # base requirements

In [ ]:
!python /content/yolov5/models/export.py --weights /content/yolov5l.pt --img 640 --batch 1  # export at 640x640 with batch size 1

Namespace(batch_size=1, device='cpu', dynamic=False, grid=False, img_size=[640, 640], weights='/content/yolov5l.pt')
YOLOv5 v4.0-126-g886f1c0 torch 1.8.0+cu101 CPU

Fusing layers... 
Model Summary: 392 layers, 47025981 parameters, 0 gradients, 115.4 GFLOPS

Starting TorchScript export with torch 1.8.0+cu101...
/usr/local/lib/python3.7/dist-packages/torch/jit/_trace.py:940: TracerWarning: Encountering a list at the output of the tracer might cause the trace to be incorrect, this is only valid if the container structure does not change based on the module's inputs. Consider using a constant container instead (e.g. for `list`, use a `tuple` instead. for `dict`, use a `NamedTuple` instead). If you absolutely need this and know the side effects, pass strict=False to trace() to allow this behavior.
  _force_outplace,
TorchScript export success, saved as /content/yolov5l.torchscript.pt

Starting ONNX export with onnx 1.8.1...
ONNX export success, saved as /content/yolov5l.onnx
scikit-learn ve

In [ ]:
%cd /content/yolov5/

/content/yolov5


In [ ]:
!cp /content/yolov5l.onnx /content/drive/Shareddrives/AI

In [ ]:
%cd /content/

/content


In [ ]:
!pip install git+https://github.com/onnx/onnx-tensorflow.git

  Cloning https://github.com/onnx/onnx-tensorflow.git to /tmp/pip-req-build-vkkt29wv
  Running command git clone -q https://github.com/onnx/onnx-tensorflow.git /tmp/pip-req-build-vkkt29wv
     |████████████████████████████████| 706kB 11.2MB/s 
  Created wheel for onnx-tf: filename=onnx_tf-1.7.0-cp37-none-any.whl size=218776 sha256=f99a82cf3dbf9fd410dae6c6a979fd2736bc55ac4236b6c8498a3180d5743b12
  Stored in directory: /tmp/pip-ephem-wheel-cache-y5l3_1yq/wheels/54/24/31/8873b33d2d560efdfa1ed6f346df67ef793b1897358705a480
Successfully built onnx-tf


In [ ]:
import onnx
from tensorflow.python.tools.import_pb_to_tensorboard import import_to_tensorboard
from onnx_tf.backend import prepare
onnx_model = onnx.load("/content/yolov5l.onnx")
tf_rep = prepare(onnx_model)
tf_rep.export_graph("best.pb")
import_to_tensorboard("best.pb", "tb_log")

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 1.15.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,


ImportError: ignored

In [ ]:
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow

Cloning into 'onnx-tensorflow'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 6073 (delta 8), reused 8 (delta 4), pack-reused 6051
Receiving objects: 100% (6073/6073), 1.84 MiB | 20.93 MiB/s, done.
Resolving deltas: 100% (4733/4733), done.


In [ ]:
!pip install yapf
!python yapf -rip --style="{based_on_style: google, indent_width: 2}" $FilePath$

python3: can't open file 'yapf': [Errno 2] No such file or directory


In [ ]:
%cd /content/onnx-tensorflow

/content/onnx-tensorflow


In [ ]:
!git clone https://github.com/tensorflow/addons.git

Cloning into 'addons'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 12096 (delta 12), reused 10 (delta 4), pack-reused 12057
Receiving objects: 100% (12096/12096), 5.55 MiB | 5.67 MiB/s, done.
Resolving deltas: 100% (8749/8749), done.


In [ ]:
!python /content/onnx-tensorflow/example/onnx_to_tf.py

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 1.15.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,
Traceback (most recent call last):
  File "/content/onnx-tensorflow/example/onnx_to_tf.py", line 3, in <module>
    from onnx_tf.backend import prepare
  File "/usr/local/lib/python3.7/dist-packages/onnx_tf/__init__.py", line 1, in <module>
    from . import backend
  File "/usr/loc

In [ ]:
!python /content/onnx-tensorflow/onnx_tf/converter.py -i /content/yolov5l.onnx -o /content/output.pb

/usr/local/lib/python3.7/dist-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.3.0 and strictly below 2.5.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 1.15.2 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,
Traceback (most recent call last):
  File "/content/onnx-tensorflow/onnx_tf/converter.py", line 12, in <module>
    import onnx_tf.backend as backend
  File "/usr/local/lib/python3.7/dist-packages/onnx_tf/__init__.py", line 1, in <module>
    from . import backend
  File "/usr/local

In [ ]:
!python /content/onnx-tensorflow/setup.py

usage: setup.py [global_opts] cmd1 [cmd1_opts] [cmd2 [cmd2_opts] ...]
   or: setup.py --help [cmd1 cmd2 ...]
   or: setup.py --help-commands
   or: setup.py cmd --help

error: no commands supplied
